In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
from pathlib import Path
from glob import glob

import cv2
import skimage
import matplotlib.pyplot as plt

BACKEND = Path(os.getcwd()).parent
sys.path.append(str(BACKEND))
from inference_model import _init_model, mmdet_to_coco, coco_to_csv, coco_to_mmdet

/home/badboy-002/anaconda3/envs/senior_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
demo_ann = BACKEND / 'demo_files' / 'result_demo.json'
with open(demo_ann, 'r') as f:
    demo_ann = json.load(f)
print(demo_ann)

[]


In [2]:
import os
import tensorflow as tf

2023-04-27 20:28:44.441659: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [1]:
import os
import tensorflow as tf

def prune_model(model_path):
    # Load the model using tf.keras.models.load_model()
    model = tf.keras.models.load_model(model_path)

    # Prune the model (for example, let's prune all dropout layers)
    pruned_model = tf.keras.models.Sequential()
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Dropout):
            continue
        pruned_model.add(layer)

    # Save the pruned model to a new .h5 file in the same folder as the original model
    model_folder, model_filename = os.path.split(model_path)
    pruned_model_path = os.path.join(model_folder, f"pruned_{model_filename}")

    pruned_model.save(pruned_model_path)

    print(f"Pruned model saved to: {pruned_model_path}")

    return pruned_model_path


2023-04-27 16:56:25.907459: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
model = tf.keras.models.load_model('/home/badboy-002/github/senior_project/web-app/deepmac_1024x1024_coco17/saved_model')

2023-04-27 20:28:49.011067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-27 20:28:49.026114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-27 20:28:49.026254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-27 20:28:49.026636: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [4]:
type(model)

tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject

In [ ]:
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude